In [11]:
import torch

labels = torch.tensor([1, 2, -1, -1])
sim = torch.rand(4, 4)
global_R = torch.full((labels.size(0), labels.size(0)), -1.0)
mask_label = (labels != -1)
label_mask = mask_label.unsqueeze(0) & mask_label.unsqueeze(1)
label_R = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
global_R[label_mask] = label_R[label_mask]

print(global_R)

tensor([[ 1.,  0., -1., -1.],
        [ 0.,  1., -1., -1.],
        [-1., -1., -1., -1.],
        [-1., -1., -1., -1.]])


In [12]:
other_mask = ~label_mask
l, u = 0.5, 0.9
global_R[other_mask & (sim >= u)] = torch.tensor(1.0)
global_R[other_mask & (sim <= l)] = torch.tensor(0.0)
print(sim)
print(global_R)

tensor([[0.1877, 0.4130, 0.5019, 0.4840],
        [0.1811, 0.6732, 0.4352, 0.5160],
        [0.1859, 0.0105, 0.9525, 0.8025],
        [0.5501, 0.6766, 0.8671, 0.9637]])
tensor([[ 1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.],
        [ 0.,  0.,  1., -1.],
        [-1., -1., -1.,  1.]])


In [14]:
uncert_mask = (global_R == -1)
mask = torch.tril(uncert_mask, diagonal=-1)
row, col = torch.where(mask)
uncert_ij = torch.stack([row, col], dim=1).tolist()
print(uncert_ij)

[[3, 0], [3, 1], [3, 2]]
